In [1]:
#importing necessary libraries 
import pandas as pd
import hvplot.pandas
import numpy as np
import json
import csv 
import matplotlib.pyplot as plt
import requests
import scipy.stats as st
import os

from api_keys import geoapify_key # importing api key

# importing the csv file
# csvpath = os.path.join("traffic-crashes-1 (2).csv")

# with open(csvpath) as csvfile:
#     csvreader = csv.reader(csvfile, delimiter = ",")
#     csvheader = next(csvreader) 

traffic_data = pd.read_csv("traffic-crashes-1 (2).csv")
traffic_data.columns


Index(['FID_1', 'OBJECTID', 'Crash_Seve', 'County', 'Crash_Date', 'Crash_Year',
       'Crash_Time', 'Primary_St', 'Distance', 'Dir', 'Secondary_', 'Weather',
       'Fatalities', 'Injured', 'Property_D', 'Injury_Typ', 'Crash_Type',
       'Total_Vehi', 'V1_Type', 'V1_Dir', 'V1_Driver_', 'V1_Lane_Nu',
       'V1_Action', 'V1_Driver1', 'V1_Drive_1', 'V1_Vehicle', 'V1_Most_Ha',
       'V1_All_Eve', 'V2_Type', 'V2_Dir', 'V2_Driver_', 'V2_Lane_Nu',
       'V2_Action', 'V2_Driver1', 'V2_Drive_1', 'V2_Vehicle', 'V2_Most_Ha',
       'V2_All_Eve', 'First_Harm', 'Nonmotoris', 'Factors_Ro', 'Lighting',
       'HWY_Factor', 'Agency', 'Accident_R', 'Pedalcycli', 'Pedestrian',
       'Motorcycli', 'AnimalType', 'X', 'Y', 'OBJECTID_1', 'AREA', 'PERIMETER',
       'WARD', 'CNTY_IS', 'SHADE', 'ACRES', 'SQ_MILES', 'AREA_1', 'LEN'],
      dtype='object')

In [2]:
traffic_data[["County","Crash_Seve"]]

,County,Crash_Seve
0,CLARK,PROPERTY DAMAGE ONLY
1,CLARK,INJURY CRASH
2,CLARK,PROPERTY DAMAGE ONLY
3,CLARK,PROPERTY DAMAGE ONLY
4,CLARK,PROPERTY DAMAGE ONLY
...,...,...
12715,CLARK,PROPERTY DAMAGE ONLY
12716,CLARK,INJURY CRASH
12717,CLARK,INJURY CRASH
12718,CLARK,INJURY CRASH


In [3]:
clean_traffic_data = traffic_data[["County",
                                   "Fatalities",
                                   "Injured",
                                   "Primary_St",
                                   "Crash_Seve",
                                   "Crash_Date",
                                   "Weather",
                                   "Crash_Type",
                                   "V1_Driver1",
                                   "V1_Driver_",
                                   "Factors_Ro",
                                   "Lighting",
                                   "X",
                                   "Y"]].copy()
clean_traffic_data.head(5)

,County,Fatalities,Injured,Primary_St,Crash_Seve,Crash_Date,Weather,Crash_Type,V1_Driver1,V1_Driver_,Factors_Ro,Lighting,X,Y
0,CLARK,NaN,0,ECHELON POINT DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,REAR-END,APPARENTLY NORMAL,49,DRY,DAYLIGHT,-115.296856,36.282588
1,CLARK,NaN,1,GRAND CANYON DR,INJURY CRASH,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - NO LIGHTING,-115.305960,36.313917
2,CLARK,NaN,0,SKY POINTE DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - CONTINUOUS LIGHTING,-115.266269,36.279880
3,CLARK,NaN,0,CC215S,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,NaN,0,WET,DARK - SPOT LIGHTING,-115.283121,36.278732
4,CLARK,NaN,0,US95N,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,APPARENTLY NORMAL,27,DRY,DARK - NO LIGHTING,-115.312353,36.327534


In [4]:
clean_traffic_data["Fatalities"] = clean_traffic_data["Fatalities"].fillna(0)
len(clean_traffic_data)

12720

In [5]:
clean_traffic_data = clean_traffic_data.rename(columns={"Crash_Seve": "Crash Severity",
                                                        "Crash_Date": "Crash Date",
                                                        "Crash_Type": "Crash Type",
                                                        "V1_Driver1":"Impairment Level",
                                                        "V1_Driver_": "Age",
                                                        "Factors_Ro": "Road Conditions",
                                                        "X": "Latitude",
                                                        "Y": "Longitude"})
clean_traffic_data.head(3)


,County,Fatalities,Injured,Primary_St,Crash Severity,Crash Date,Weather,Crash Type,Impairment Level,Age,Road Conditions,Lighting,Latitude,Longitude
0,CLARK,0.0,0,ECHELON POINT DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,REAR-END,APPARENTLY NORMAL,49,DRY,DAYLIGHT,-115.296856,36.282588
1,CLARK,0.0,1,GRAND CANYON DR,INJURY CRASH,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - NO LIGHTING,-115.305960,36.313917
2,CLARK,0.0,0,SKY POINTE DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - CONTINUOUS LIGHTING,-115.266269,36.279880


In [6]:
clean_traffic_data = clean_traffic_data.dropna()
clean_traffic_data.head(5)

,County,Fatalities,Injured,Primary_St,Crash Severity,Crash Date,Weather,Crash Type,Impairment Level,Age,Road Conditions,Lighting,Latitude,Longitude
0,CLARK,0.0,0,ECHELON POINT DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,REAR-END,APPARENTLY NORMAL,49,DRY,DAYLIGHT,-115.296856,36.282588
1,CLARK,0.0,1,GRAND CANYON DR,INJURY CRASH,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - NO LIGHTING,-115.305960,36.313917
2,CLARK,0.0,0,SKY POINTE DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - CONTINUOUS LIGHTING,-115.266269,36.279880
4,CLARK,0.0,0,US95N,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,APPARENTLY NORMAL,27,DRY,DARK - NO LIGHTING,-115.312353,36.327534
6,CLARK,0.0,0,GASS AVE,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,ANGLE,HAD BEEN DRINKING: OTHER IMPROPER DRIVING,26,DRY,DARK - CONTINUOUS LIGHTING,-115.146042,36.160959


In [7]:
#Day or evening occurance
crash_stats_mean = clean_traffic_data.groupby("Crash Type")["Age"].mean()
crash_stats_med = clean_traffic_data.groupby("Crash Type")["Age"].median()
crash_stats_var = clean_traffic_data.groupby("Crash Type")["Age"].var()
crash_stats_std = clean_traffic_data.groupby("Crash Type")["Age"].std()
crash_stats_sem = clean_traffic_data.groupby("Crash Type")["Age"].sem()

age_stats = pd.DataFrame({"Mean age":crash_stats_mean,"Med age":crash_stats_med,"Var age":crash_stats_var,"Std age":crash_stats_std,"Sem age":crash_stats_sem})
age_stats


,Mean age,Med age,Var age,Std age,Sem age
Crash Type,,,,,
ANGLE,39.060997,35.0,340.981318,18.465679,0.316219
BACKING,42.213115,42.0,378.970492,19.467164,2.492515
HEAD-ON,35.146341,29.0,357.478049,18.907090,2.952791
NON-COLLISION,34.827964,30.0,246.508747,15.700597,0.398539
REAR-END,35.026336,31.0,237.293728,15.404341,0.247525
REAR-TO-REAR,34.695652,34.0,161.675889,12.715183,2.651299
"SIDESWIPE, MEETING",40.893805,40.0,337.042193,18.358709,1.727042
"SIDESWIPE, OVERTAKING",38.719551,37.0,256.593774,16.018545,0.641255
UNKNOWN,36.352941,37.0,333.617647,18.265203,4.429962


In [8]:
age_stats_agg = clean_traffic_data.groupby("Crash Type")[["Age"]].agg(["mean","median","var","std","sem"])
round(age_stats_agg,2)

Age                            
                        mean median     var    std   sem
Crash Type                                              
ANGLE                  39.06   35.0  340.98  18.47  0.32
BACKING                42.21   42.0  378.97  19.47  2.49
HEAD-ON                35.15   29.0  357.48  18.91  2.95
NON-COLLISION          34.83   30.0  246.51  15.70  0.40
REAR-END               35.03   31.0  237.29  15.40  0.25
REAR-TO-REAR           34.70   34.0  161.68  12.72  2.65
SIDESWIPE, MEETING     40.89   40.0  337.04  18.36  1.73
SIDESWIPE, OVERTAKING  38.72   37.0  256.59  16.02  0.64
UNKNOWN                36.35   37.0  333.62  18.27  4.43

In [9]:
Weather_by_age = clean_traffic_data.groupby("Weather")["Age"].agg(["mean","median","var","std","sem"])
round(Weather_by_age,2)

,mean,median,var,std,sem
Weather,,,,,
"BLOWING SAND, SOIL, DIRT, SNOW",38.56,35.0,184.53,13.58,4.53
CLEAR,36.75,33.0,283.70,16.84,0.19
CLOUDY,37.40,34.0,299.57,17.31,0.47
"CLOUDY: BLOWING SAND, SOIL, DIRT, SNOW",35.00,35.0,162.00,12.73,9.00
"CLOUDY: FOG, SMOG, SMOKE",20.00,20.0,NaN,NaN,NaN
CLOUDY: RAIN,34.91,31.0,249.27,15.79,1.32
CLOUDY: SEVERE CROSSWINDS,33.67,32.0,184.33,13.58,7.84
"FOG, SMOG, SMOKE",42.25,44.5,377.58,19.43,9.72
OTHER,26.33,22.0,124.33,11.15,6.44


In [10]:
Impairment_by_age = clean_traffic_data.groupby("Impairment Level")["Age"].agg(["mean","median","var","std","sem"])
round(Impairment_by_age,2)

,mean,median,var,std,sem
Impairment Level,,,,,
APPARENTLY NORMAL,37.28,33.0,286.50,16.93,0.19
DRUG INVOLVEMENT,34.37,31.0,190.75,13.81,1.16
DRUG INVOLVEMENT: OTHER IMPROPER DRIVING,53.50,53.5,312.50,17.68,12.50
"FELL ASLEEP, FAINTED, FATIGUED, ETC.",33.86,27.5,269.25,16.41,1.50
"FELL ASLEEP, FAINTED, FATIGUED, ETC.: DRUG INVOLVEMENT",29.83,25.0,161.37,12.70,5.19
"FELL ASLEEP, FAINTED, FATIGUED, ETC.: HAD BEEN DRINKING",31.46,24.0,131.10,11.45,3.18
"FELL ASLEEP, FAINTED, FATIGUED, ETC.: HAD BEEN DRINKING: DRUG INVOLVEMENT",21.00,21.0,NaN,NaN,NaN
"FELL ASLEEP, FAINTED, FATIGUED, ETC.: OTHER IMPROPER DRIVING",51.00,51.0,578.00,24.04,17.00
HAD BEEN DRINKING,33.16,30.0,158.36,12.58,0.49


In [11]:
Road_by_age = clean_traffic_data.groupby("Road Conditions")["Age"].agg(["mean","median","var","std","sem"])
Road_by_age = round(Road_by_age,2)
Road_by_age = Road_by_age.drop(labels=["ICE","OTHER"],axis=0,inplace=False)
Road_by_age


,mean,median,var,std,sem
Road Conditions,,,,,
DRY,36.94,33.0,287.85,16.97,0.18
WET,35.06,31.0,232.94,15.26,0.51


In [12]:
accident_lighting = clean_traffic_data.groupby(["Lighting", "Weather"])["Lighting"].count()
accident_lighting



Lighting                    Weather                               
DARK - CONTINUOUS LIGHTING  CLEAR                                      999
                            CLOUDY                                     117
                            CLOUDY: RAIN                                18
                            OTHER                                        1
                            RAIN                                        28
                            SEVERE CROSSWINDS                            1
DARK - NO LIGHTING          BLOWING SAND, SOIL, DIRT, SNOW               1
                            CLEAR                                       76
                            CLOUDY                                      10
                            RAIN                                         9
DARK - SPOT LIGHTING        BLOWING SAND, SOIL, DIRT, SNOW               1
                            CLEAR                                     1288
                            CLOUD

In [13]:
# accident_lighting = accident_lighting.drop(labels=["OTHER","UNKNOWN"],axis=0,inplace=False)
# accident_lighting
#removed other and unknown

In [14]:
clean_traffic_data["day_night"]= clean_traffic_data["Lighting"].str[: 3]
clean_traffic_data

,County,Fatalities,Injured,Primary_St,Crash Severity,Crash Date,Weather,Crash Type,Impairment Level,Age,Road Conditions,Lighting,Latitude,Longitude,day_night
0,CLARK,0.0,0,ECHELON POINT DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,REAR-END,APPARENTLY NORMAL,49,DRY,DAYLIGHT,-115.296856,36.282588,DAY
1,CLARK,0.0,1,GRAND CANYON DR,INJURY CRASH,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - NO LIGHTING,-115.305960,36.313917,DAR
2,CLARK,0.0,0,SKY POINTE DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - CONTINUOUS LIGHTING,-115.266269,36.279880,DAR
4,CLARK,0.0,0,US95N,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,APPARENTLY NORMAL,27,DRY,DARK - NO LIGHTING,-115.312353,36.327534,DAR
6,CLARK,0.0,0,GASS AVE,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,ANGLE,HAD BEEN DRINKING: OTHER IMPROPER DRIVING,26,DRY,DARK - CONTINUOUS LIGHTING,-115.146042,36.160959,DAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12714,CLARK,0.0,0,SR599N TO ANN RD INT90B,PROPERTY DAMAGE ONLY,12/31/2016,CLOUDY,REAR-END,APPARENTLY NORMAL,38,DRY,DAYLIGHT,-115.252436,36.262845,DAY
12715,CLARK,0.0,0,N BUFFALO DR,PROPERTY DAMAGE ONLY,12/31/2016,CLOUDY,NON-COLLISION,HAD BEEN DRINKING,24,DRY,DARK - CONTINUOUS LIGHTING,-115.260731,36.203052,DAR
12716,CLARK,0.0,1,CLIFF SHADOWS PKWY,INJURY CRASH,12/31/2016,CLEAR,NON-COLLISION,"FELL ASLEEP, FAINTED, FATIGUED, ETC.",68,DRY,DAYLIGHT,-115.329666,36.239294,DAY
12717,CLARK,0.0,3,CHEYENNE AVE,INJURY CRASH,12/31/2016,CLOUDY,REAR-END,APPARENTLY NORMAL,65,DRY,DARK - CONTINUOUS LIGHTING,-115.322318,36.218215,DAR


In [15]:
accident_lighting = clean_traffic_data.groupby(["day_night", "Weather"])["day_night"].count()
accident_lighting

day_night  Weather                               
DAR        BLOWING SAND, SOIL, DIRT, SNOW               2
           CLEAR                                     2368
           CLOUDY                                     314
           CLOUDY: RAIN                                43
           CLOUDY: SEVERE CROSSWINDS                    1
           FOG, SMOG, SMOKE                             2
           OTHER                                        2
           RAIN                                       164
           SEVERE CROSSWINDS                            3
           UNKNOWN                                      6
DAW        CLEAR                                      125
           CLOUDY                                      11
           CLOUDY: RAIN                                 3
           RAIN                                         8
DAY        BLOWING SAND, SOIL, DIRT, SNOW               7
           CLEAR                                     5247
           CLOUDY     

In [16]:
accident_lighting = accident_lighting.drop(labels=["OTH","UNK"],axis=0,inplace=False)
accident_lighting

day_night  Weather                               
DAR        BLOWING SAND, SOIL, DIRT, SNOW               2
           CLEAR                                     2368
           CLOUDY                                     314
           CLOUDY: RAIN                                43
           CLOUDY: SEVERE CROSSWINDS                    1
           FOG, SMOG, SMOKE                             2
           OTHER                                        2
           RAIN                                       164
           SEVERE CROSSWINDS                            3
           UNKNOWN                                      6
DAW        CLEAR                                      125
           CLOUDY                                      11
           CLOUDY: RAIN                                 3
           RAIN                                         8
DAY        BLOWING SAND, SOIL, DIRT, SNOW               7
           CLEAR                                     5247
           CLOUDY     

In [17]:
accident_int = clean_traffic_data.groupby(["Primary_St"])["Primary_St"].value_counts()
accident_int 

Primary_St
28TH ST                2
30TH ST                1
ALPINE PL              1
ALTA DR               61
AMY OLIVIA AVE         1
                      ..
WINDROSE POINT AVE     1
WING ST                1
WOODBRIDGE DR          1
WOODLAND RIDGE ST      1
WYANDOTTE ST           1
Name: count, Length: 648, dtype: int64

In [18]:
accident_int = pd.DataFrame(accident_int)
accident_int = accident_int.sort_values("count", ascending=False)
accident_int.head(10)

,count
Primary_St,
IR15N,961
US95S,700
US95N,597
SR159,436
IR15S,435
SR589,226
LAKE MEAD BLVD,191
SR599,174
EASTERN AVE,153


In [19]:
accident_street_df = clean_traffic_data[[ "County", "Age", "Latitude","Longitude", "Primary_St"]].copy()
accident_street_df

,County,Age,Latitude,Longitude,Primary_St
0,CLARK,49,-115.296856,36.282588,ECHELON POINT DR
1,CLARK,22,-115.305960,36.313917,GRAND CANYON DR
2,CLARK,22,-115.266269,36.279880,SKY POINTE DR
4,CLARK,27,-115.312353,36.327534,US95N
6,CLARK,26,-115.146042,36.160959,GASS AVE
...,...,...,...,...,...
12714,CLARK,38,-115.252436,36.262845,SR599N TO ANN RD INT90B
12715,CLARK,24,-115.260731,36.203052,N BUFFALO DR
12716,CLARK,68,-115.329666,36.239294,CLIFF SHADOWS PKWY
12717,CLARK,65,-115.322318,36.218215,CHEYENNE AVE


### Narrow down the accident_street_df to show top 10 streets that had accidents

In [20]:
clean_accident_data = pd.merge(accident_street_df, accident_int, how="left", on=["Primary_St", "Primary_St"])
clean_accident_data = pd.DataFrame(clean_accident_data)
clean_accident_data

,County,Age,Latitude,Longitude,Primary_St,count
0,CLARK,49,-115.296856,36.282588,ECHELON POINT DR,1
1,CLARK,22,-115.305960,36.313917,GRAND CANYON DR,14
2,CLARK,22,-115.266269,36.279880,SKY POINTE DR,20
3,CLARK,27,-115.312353,36.327534,US95N,597
4,CLARK,26,-115.146042,36.160959,GASS AVE,9
...,...,...,...,...,...,...
9709,CLARK,38,-115.252436,36.262845,SR599N TO ANN RD INT90B,20
9710,CLARK,24,-115.260731,36.203052,N BUFFALO DR,98
9711,CLARK,68,-115.329666,36.239294,CLIFF SHADOWS PKWY,11
9712,CLARK,65,-115.322318,36.218215,CHEYENNE AVE,103


In [21]:
top_street_df = clean_accident_data.loc[(clean_accident_data["count"] >= 149)]
top_street_df = pd.DataFrame(top_street_df)
top_street_df

,County,Age,Latitude,Longitude,Primary_St,count
3,CLARK,27,-115.312353,36.327534,US95N,597
6,CLARK,23,-115.165380,36.150421,IR15N,961
8,CLARK,22,-115.166759,36.148856,IR15N,961
9,CLARK,46,-115.163842,36.152167,IR15N,961
10,CLARK,25,-115.165478,36.150309,IR15N,961
...,...,...,...,...,...,...
9696,CLARK,35,-115.115841,36.167172,EASTERN AVE,153
9700,CLARK,36,-115.173848,36.140799,IR15N,961
9702,CLARK,28,-115.175425,36.139011,IR15N,961
9703,CLARK,39,-115.174834,36.139682,IR15N,961


In [22]:
# Set parameters to search for a street

# params = {
#     "apiKey": geoapify_key,
#     "mode": "drive"
#     }

# for index, row in top_street_df.iterrows():
#     #get latidude and longitude from DataRFrame
#     lat = row["Latitude"]
#     lng = row ["Longitude"]

#     # Add waypoints current street's latitude and longitude to the params dictionary
#     params["waypoints"] = f"circle:{lng},{lat}"

# base_url = " https://api.geoapify.com/v1/mapmatching"

# # Make and API request using the params dictionaty
# street_address = requests.get(base_url, params=params)
    
# # Convert the API response to JSON format
# street_address = street_address.json()
    
# # Display sample data
# clean_accident_data

In [23]:
accidents = top_street_df.hvplot.points(
    y="Longitude", 
    x="Latitude",
    geo = True,
    width=800,
    height=400,
    color="Primary_St",
    size="Age",
    tiles="OSM",
    hover_cols = ["Primary_St", "County", "Age"])

accidents

c:\Users\User\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Latitude,Longitude]   (Primary_St,Age,County)